In [1]:
import numpy as np
import pandas as pd
import json
from pprint import pprint

In [2]:
import time

In [3]:
import tensorflow as tf

In [4]:
import os

Далее используется текст, сформированный из моей личной переписки в телеграме. Знак доллара - токен конца сообщения.

In [5]:
with open('Whole_text.txt', 'rb') as f:
    text = f.read().decode('utf-8')

In [6]:
text[:100]

'Курить пойдёшь? $ Бабаську надо сначала отдать $ Она же ещё не готова $ Уже готова пол минуты для за'

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

371 unique characters


In [8]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None, encoding='utf-8')

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None, encoding='utf-8')

In [10]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [11]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, input_encoding='UTF-8'))
all_ids

<tf.Tensor: shape=(1101940,), dtype=int64, numpy=array([116, 156, 153, ..., 142, 142,  17], dtype=int64)>

In [12]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [13]:
for ids in ids_dataset.take(50):
    print(chars_from_ids(ids).numpy().decode('utf-8'), end='')

Курить пойдёшь? $ Бабаську надо сначала отдать $ О

In [14]:
seq_length = 100

In [15]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
dataset = sequences.map(split_input_target)

## Create training batches

In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [19]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [22]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
          states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [23]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

### Try the model

In [24]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 372) # (batch_size, sequence_length, vocab_size)


In [25]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  95232     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  381300    
                                                                 
Total params: 4,414,836
Trainable params: 4,414,836
Non-trainable params: 0
_________________________________________________________________


In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [27]:
sampled_indices

array([ 54, 249, 104, 131, 234, 149,  76,  73, 334,  19, 339,  30, 237,
        31,  89, 108, 195, 349, 118, 226,  79, 110, 141,  41, 316, 166,
       173, 246, 292,  98, 359, 151, 234, 123, 235, 184, 124, 207,  61,
       106, 343, 228, 331, 156, 164,   1, 197, 282,  80, 141, 361,  33,
       337, 195, 139, 363, 219, 149, 278,  76,  65, 144,  91, 204, 308,
        73, 137, 277, 162,  86,  71,  21,  30, 180, 131, 188, 288, 200,
       315, 113, 366,  18,  65, 324, 142, 330, 140, 176, 168, 349,  78,
       368, 217, 338, 355,  45, 167, 371, 265, 278], dtype=int64)

In [28]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [29]:
model.compile(optimizer='adam', loss=loss)

In [28]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [126]:
EPOCHS = 20

In [140]:
history = model.fit(dataset, epochs=30, callbacks=[checkpoint_callback], initial_epoch=20, workers=6, use_multiprocessing=True)

Epoch 21/30
170/170 [==============================] - 403s 2s/step - loss: 0.7741
Epoch 22/30
170/170 [==============================] - 484s 3s/step - loss: 0.7322
Epoch 23/30
170/170 [==============================] - 447s 3s/step - loss: 0.6935
Epoch 24/30
170/170 [==============================] - 456s 3s/step - loss: 0.6640
Epoch 25/30
170/170 [==============================] - 486s 3s/step - loss: 0.6366
Epoch 26/30
170/170 [==============================] - 486s 3s/step - loss: 0.6121
Epoch 27/30
170/170 [==============================] - 486s 3s/step - loss: 0.5912
Epoch 28/30
170/170 [==============================] - 482s 3s/step - loss: 0.5732
Epoch 29/30
170/170 [==============================] - 481s 3s/step - loss: 0.5606
Epoch 30/30
170/170 [==============================] - 483s 3s/step - loss: 0.5465


In [55]:
tf.train.latest_checkpoint('training_checkpoints')

'training_checkpoints\\ckpt_30'

In [42]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [43]:
model.compile(optimizer='adam', loss=loss)

In [56]:
model.load_weights('training_checkpoints/ckpt_30')

In [59]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback], initial_epoch=30, workers=6, use_multiprocessing=True)

Epoch 31/100
170/170 [==============================] - 497s 3s/step - loss: 2.7583
Epoch 32/100
170/170 [==============================] - 499s 3s/step - loss: 1.8798
Epoch 33/100
170/170 [==============================] - 566s 3s/step - loss: 1.7566
Epoch 34/100
170/170 [==============================] - 543s 3s/step - loss: 1.6825
Epoch 35/100
170/170 [==============================] - 521s 3s/step - loss: 1.6294
Epoch 36/100
170/170 [==============================] - 499s 3s/step - loss: 1.5860
Epoch 37/100
170/170 [==============================] - 522s 3s/step - loss: 1.5516
Epoch 38/100
170/170 [==============================] - 544s 3s/step - loss: 1.5202
Epoch 39/100
170/170 [==============================] - 545s 3s/step - loss: 1.4935
Epoch 40/100
170/170 [==============================] - 544s 3s/step - loss: 1.4669
Epoch 41/100
170/170 [==============================] - 511s 3s/step - loss: 1.4436
Epoch 42/100
170/170 [==============================] - 516s 3s/step - loss:

In [60]:
history = model.fit(dataset, epochs=200, callbacks=[checkpoint_callback], initial_epoch=100, workers=6, use_multiprocessing=True)

Epoch 101/200
170/170 [==============================] - 427s 3s/step - loss: 0.7167
Epoch 102/200
170/170 [==============================] - 424s 2s/step - loss: 0.7168
Epoch 103/200
170/170 [==============================] - 409s 2s/step - loss: 0.7121
Epoch 104/200
170/170 [==============================] - 410s 2s/step - loss: 0.7053
Epoch 105/200
170/170 [==============================] - 410s 2s/step - loss: 0.7033
Epoch 106/200
170/170 [==============================] - 414s 2s/step - loss: 0.6937
Epoch 107/200
170/170 [==============================] - 410s 2s/step - loss: 0.6907
Epoch 108/200
170/170 [==============================] - 410s 2s/step - loss: 0.6910
Epoch 109/200
170/170 [==============================] - 414s 2s/step - loss: 0.6888
Epoch 110/200
170/170 [==============================] - 415s 2s/step - loss: 0.6844
Epoch 111/200
170/170 [==============================] - 415s 2s/step - loss: 0.6798
Epoch 112/200
170/170 [==============================] - 413s 2s/

In [61]:
model.save_weights('./training_checkpoints/my_checkpoint')

In [30]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
        # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
    @tf.function
    def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [31]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=1.0)

Предсказание необученной модели

In [32]:
start = time.time()
states = None
next_char = tf.constant(['Привет, как дела?'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Привет, как дела?🐴Д😋f💋L|П😙🫥😡🍒]😖Ж😆t🥞🍷щС😗.,🤢😿SО🤔щ🥞№жЗ😅🍷3🐖I😌😎SГ🫣🤯🔺🙏🥺🍛💩😜ë🤢🙁🤭cЬ👯🥗👇v🤷🥞н🥵✈;≤😊 
1❗🦁З🥺🦠п❤🤩pGw👈🥗иb₽C_т🍑💋🚲С9у🧡/p]Ч🦁🤨😩😁🤩vНвt🙂aг🤜ЛИUg’\9,ЖЧ9‘L℅😷Ф℅🤯Ё!oэ🧠w😞🐷😘Г🎤😻🥳😡№😛F😯V😉≤🥺#.🤪ЩЕU🐷PL🥩😡юЁA–🫣ц3☝$⠀🥳—😚😂😽m🕐🥂🥴😇🏻⚡💆Ё🎵p℅😇🕐❗🏥🐱Ы😕🐴Н😖😩😢F🤮🥹sAЩ🫥😁]😊
😸👍🐱🫣💪♂🧚n:🤗féж🐖🍪😀😸й₽щ🐅:♻😄;l🍊₽😅😃Л👌🧃🌧💕ш]=🥰В0🍕ё•🎅🍑6В🙀✅μЯ🏥🕐m🙏🐐НЬJ…℅🎵,😐🥵😹 'М🤭и☝Ц🐱😵]К🙇3Йv💪👍Ш⚡i😲🥗—😴🍞Jt℅✈🚗🕐&😨😣K9Хj5"шС%🌈😙🍋.😸😩💕☔🤣👈йО2😵°I🙀🤯🍞♂🤑🍀«👆🦠Dъ👯пn😯🤪🙂😞😋🥳R🌧й🫣😀-C'Сë🏥ю🤯¡😔🥶Фl😴😲О🎈b😚🙏🫶Ч🐑ЯР😀🦜🎈l🫥🔺🤯🤷ЧБ]b😢)‘=АS👈🍦🙏😨ДPх😀Ф[КФ]😴🫶D🥶👎o🙇юq😚🤕🔥😵M😛🍑о💐]l❤.:K🦁
хкО😆h🙌😕🦁!🏼h😬🙀яБ¡👌Йa➦😖Кxz🧠ж🐴♻😜☝GСыГ🐴з👀😗😡😀l👻😲Т🍷♻😽н🍪℅B|🎵🦜NЙD#ю🤑🏼М👍c🍀👇Я🙇I😐fшК🦁ЛqП…✌«M3💔🙌В👍—🥴SNйУ🪙😒👍🥴цйj😢💯$🍓👆😂💛🤯0ob*thP°💐с🤗JЗ🎵🤩🤯🙏😱🌹3🐖😂🙁♻🏻f🐐г🐴/EГ
👎🐖💛ю#🤫j😖😉' ЮЯ‘🤪💩dës💕♂) 

________________________________________________________________________________

Run time: 8.070864915847778


In [35]:
model.load_weights('./training_checkpoints/my_checkpoint')

Предсказание обученной модели

In [66]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars, temperature=0.4)

In [68]:
start = time.time()
states = None
next_char = tf.constant(['Привет, как дела?'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Привет, как дела? $ Пока сидеть что еще капец $ Приехал $ Можно, без смотри пожары, может быть очень красиво $ Прости, зайчик, заболел сидень, то и забыла на приеме ждать $ У нас тут тончательно $ Сейчас встретился ? $ Вроде бы даже на санацию $ Супер $ Ты бы видел в какой план? $ Цля не приезжай $ Только обоица. Зато на ногу сырок с димой, я даже не понимаю как ты думал помыться $ Слышайся $ Хорошо $ Напиши когда ты говорили с папуском долго добираться $ Ну после того, что состояние и надо отдыхай $ А ты где? Тебя же ела будешь твоё? $ А как на остановке?) $ Конечно $ Поедешь как Маруся? Так я погода обратно $ Мамбе привезти? $ Да все в порядке $ Спасибо ❤️ $ Шерсина сейчас пойдём 5 смайл, только что уже съела $ Супер! Спасибо ❤️ $ Можно чтобы на ноуте с тобой сегодня собирается $ Ну и сама капсулану лучше на пухольше никто не скушал? $ Нет $ Так может я только что освобожусь $ Уснула и поела и узнализон и какой то растаевчик:) $ Есть фусяк. Со мной на кухню под давно скидывает $ Лан,

Модель очевидно неплохо выучила слова, но предлодения пока не очень, стоит обучать дальше или выбрать другую архитектуру модели